In [ ]:
import numpy as np
import os
import shutil
import PIL
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Activation,Dropout,BatchNormalization, Input
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, Adamax
from sklearn.metrics import confusion_matrix, classification_report
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

### create a function to plot training data from model.fit

In [ ]:
def tr_plot(tr_data, start_epoch):
    #Plot the training and validation data
    tacc=tr_data.history['accuracy']
    tloss=tr_data.history['loss']
    vacc=tr_data.history['val_accuracy']
    vloss=tr_data.history['val_loss']
    Epoch_count=len(tacc)+ start_epoch
    Epochs=[]
    for i in range (start_epoch ,Epoch_count):
        Epochs.append(i+1)   
    index_loss=np.argmin(vloss)#  this is the epoch with the lowest validation loss
    val_lowest=vloss[index_loss]
    index_acc=np.argmax(vacc)
    acc_highest=vacc[index_acc]
    plt.style.use('fivethirtyeight')
    sc_label='best epoch= '+ str(index_loss+1 +start_epoch)
    vc_label='best epoch= '+ str(index_acc + 1+ start_epoch)
    fig,axes=plt.subplots(nrows=1, ncols=2, figsize=(20,8))
    axes[0].plot(Epochs,tloss, 'r', label='Training loss')
    axes[0].plot(Epochs,vloss,'g',label='Validation loss' )
    axes[0].scatter(index_loss+1 +start_epoch,val_lowest, s=150, c= 'blue', label=sc_label)
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[1].plot (Epochs,tacc,'r',label= 'Training Accuracy')
    axes[1].plot (Epochs,vacc,'g',label= 'Validation Accuracy')
    axes[1].scatter(index_acc+1 +start_epoch,acc_highest, s=150, c= 'blue', label=vc_label)
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    plt.tight_layout
    #plt.style.use('fivethirtyeight')
    plt.show()


### define function to print text in RGB foreground and background colors

In [ ]:
def print_in_color(txt_msg,fore_tupple,back_tupple,):
    #prints the text_msg in the foreground color specified by fore_tupple with the background specified by back_tupple 
    #text_msg is the text, fore_tupple is foregroud color tupple (r,g,b), back_tupple is background tupple (r,g,b)
    rf,gf,bf=fore_tupple
    rb,gb,bb=back_tupple
    msg='{0}' + txt_msg
    mat='\33[38;2;' + str(rf) +';' + str(gf) + ';' + str(bf) + ';48;2;' + str(rb) + ';' +str(gb) + ';' + str(bb) +'m' 
    print(msg .format(mat), flush=True)
    print('\33[0m', flush=True) # returns default print color to back to black
    return

### define the directories 

In [ ]:
sdir=r'../input/100-bird-species'
train_dir=os.path.join(sdir, 'train')
test_dir=os.path.join(sdir, 'test')
valid_dir=os.path.join(sdir, 'valid')


### create the Datasets

In [ ]:
img_shape=(128,128,3) # use 128 X128 versus 224 X224 to reduce training time
img_size=(img_shape[0], img_shape[1])
msg='For training set'
print_in_color(msg, (0,255,255), (55,65,80))
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
            train_dir, image_size=img_size, seed=123, batch_size=30)
msg=' For validation set'
print_in_color(msg, (0,255,255), (55,65,80))
valid_ds=tf.keras.preprocessing.image_dataset_from_directory(
            valid_dir, image_size=img_size, seed=123, batch_size=30)
msg='For the test set'
print_in_color(msg, (0,255,255), (55,65,80))
test_ds=tf.keras.preprocessing.image_dataset_from_directory(
            valid_dir, image_size=img_size, shuffle=False, batch_size=30) # set shuffle=False to keep file order

### show some test images- Note with shuffle=False test images remain in original order

In [ ]:
class_names=train_ds.class_names
class_count=len(class_names)
plt.figure(figsize=(20,20))
for images, labels in test_ds.take(1):
    for i in range (25):
        plt.subplot(5,5,i +1)
        img=images[i]/255  
        plt.title(class_names[labels[i]], color='blue', fontsize=12)
        plt.imshow(img)
        plt.axis('off')
    plt.show()


### create the model

In [ ]:
input=Input(shape=img_shape)
x=tf.keras.applications.EfficientNetB3(include_top=False, weights="imagenet", pooling='max')(input) 
x=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x=Dropout(rate=.4, seed=123)(x)        
output=Dense(class_count, activation='softmax')(x)
model=Model(inputs=input, outputs=output)
model.compile(Adamax(lr=.001), loss='sparse_categorical_crossentropy', metrics=['accuracy']) # note labels are integers use sparse_categorical_crossentropy

### create reduce learning rate on plateau callback and train the model

In [ ]:
rlronp=tf.keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5,  patience=1, verbose=1)
epochs=10
history=model.fit( train_ds, validation_data=valid_ds, epochs=epochs, verbose=1, callbacks=[rlronp])
  

### plot the training data

In [ ]:
tr_plot(history, 0)

### make predictions on test set, compute accuracy and create classification report

In [ ]:
ytrue=[]
for images, label in test_ds:   
    for e in label:
        ytrue.append(class_names[e]) # list of class names associated with each image file in test dataset 
ypred=[]
errors=0
count=0
preds=model.predict(test_ds, verbose=1) # predict on the test data
for i, p in enumerate(preds):
    count +=1
    index=np.argmax(p) # get index of prediction with highest probability
    klass=class_names[index] 
    ypred.append(klass)  
    if klass != ytrue[i]:
        errors +=1
acc= (count-errors)* 100/count
msg=f'there were {count-errors} correct predictions in {count} tests for an accuracy of {acc:6.2f} % '
print_in_color(msg, (0,255,255), (55,65,80)) 
ypred=np.array(ypred)
ytrue=np.array(ytrue)
clr = classification_report(ytrue, ypred, target_names=class_names)
print("Classification Report:\n----------------------\n", clr)    

## considering the training set is not balanced and we did not use image augmentation the results are good.
## could also pobably do better using the full image size of 224 X 244 but with 300 classes the training
## time would be excessive